In [2]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-01 04:19:23--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  75.6MB/s    in 86s     

2022-05-01 04:20:50 (74.5 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-01 04:21:21--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [10]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB1(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

32161792/32148312 [==============================] - 0s 0us/step


In [12]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(240, 240))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

1%, Top 1:0.752 Top 5:0.932
2%, Top 1:0.755 Top 5:0.932
3%, Top 1:0.76 Top 5:0.9313333333333333
4%, Top 1:0.757 Top 5:0.9285
5%, Top 1:0.7568 Top 5:0.9316
6%, Top 1:0.7543333333333333 Top 5:0.9293333333333333
7%, Top 1:0.7525714285714286 Top 5:0.9302857142857143
8%, Top 1:0.75175 Top 5:0.92975
9%, Top 1:0.7546666666666667 Top 5:0.9271111111111111
10%, Top 1:0.7554 Top 5:0.9268
11%, Top 1:0.7549090909090909 Top 5:0.9267272727272727
12%, Top 1:0.7545 Top 5:0.9278333333333333
13%, Top 1:0.7567692307692307 Top 5:0.9278461538461539
14%, Top 1:0.7567142857142857 Top 5:0.9277142857142857
15%, Top 1:0.7568 Top 5:0.9273333333333333
16%, Top 1:0.75625 Top 5:0.927625
17%, Top 1:0.7556470588235294 Top 5:0.9271764705882353
18%, Top 1:0.7543333333333333 Top 5:0.9266666666666666
19%, Top 1:0.7545263157894737 Top 5:0.9263157894736842
20%, Top 1:0.7546 Top 5:0.9262
21%, Top 1:0.7542857142857143 Top 5:0.9264761904761905
22%, Top 1:0.7531818181818182 Top 5:0.9254545454545454
23%, Top 1:0.7524347826086957

In [13]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:75.46199999999999%
Top 5 Accuracy:92.596%
